# 3장 신경망 시작하기


## 구성
1. 신경망의 구조
2. 케라스 소개
3. 딥러닝 컴퓨터 세팅
4. 영화 리뷰 분류
5. 뉴스 기사 분류
6. 주택 가격 예측
7. 요약

## 이 장에서 다룰 핵심 내용
* 신경망의 핵심 구성 요소
* 케라스 소개
* 딥러닝 컴퓨터 세팅
* 신경망으로 기초적인 분류와 회기 문제 풀기

---

## 3.1 신경망의 구조

신경망 훈련에는 다음 요소들이 관련되어 있습니다.
* 네트워크 (또는 모델)을 구성하는 **층**
* **입력데이터**와 그에 상응하는 **타깃**
* 학습에 사용할 피드백 신호를 정의하는 **손실함수**
* 학습 진행 방식을 결정하는 **옵티마이저**

**층** : 딥러닝의 구성단위, 하나 이상의 텐서를 입력으로 받아 하나 이상의 텐서를 출력하는 데이터 처리 모듈입니다. 어떤 조율의 층은 상태가 없지만 대부분의 경우 가중치라는 층의 상태를 갖습니다. 가중치는 경사 하강법에 의해 학습되는 하나이상의 텐서이며 여기 네트워크가 학습한 *지식* 이 담겨있습니다.

* 완전 연결 층 (Fully Connected Layer)
* 밀집 연결 층 (Dense Connected Layer) 
* 순환 층 (Recurrent Layer)
* 2D 합성곱 층 (Convolution Layer)

In [2]:
from keras import layers
from keras import models
                
### 32개의 유닛으로 밀집된 층
### 첫 번째 차원이 784인 2D텐서만 입력으로 받는 층을 만들었습니다. 이 층은 첫 번째 차원 크기가 32로 변환된 텐서를 출력할 것입니다.
layer = layers.Dense(32, input_shape=(784, ))

model = models.Sequential()
model.add(layers.Dense(32, input_shape=(784, )))
model.add(layers.Dense(10))

### 비순환 유향 그래프(Directed Acyclic Graph) = 딥러닝 모델

네트워크 구조를 정의하고 나면 두 가지를 더 선택해야합니다.
* 손실함수 (Loss Function) (목적함수 (objective function)) : 훈련하는 동안 최소화될 값입니다. 주어진 문제에 대한 성공 지표가 됩니다.
* 옵티마이저 (Optimizer) : 손실 함수를 기반으로 네트워크가 어떻게 업데이트될지 결정합니다. 특정 종류의 확률적 경사 하강법을 구현합니다.

여러개의 출력을 내는 신경망은 여러 개의 손실함수를 가질 수 있습니다. (출력 당 하나씩) 하지만 경사하강법 과정은 하나의 스칼라 손실값 기준으로 합니다. 따라서 손실이 여러개인 네트워크에서는 모든 손실이 평균을 내서 하나의 스칼라 양으로 합쳐집니다. **문제에 맞는 올바른 목적함수**를 선택하는 것은 아주 중요합니다.

다행히 분류/회기 시퀀스 예측 같은 일반적인 문제에서는 올바른 손실 함수를 선택하는 간단한 지침이 있습니다. 예를들어 2개의 클래스가 있는 분류 문제에는 이진 크로스엔트로피 (Binary Cross-entropy), 여러 개의 클래스가 있는 분류 문제에는 범주형 크로스 엔트로피 (Categorical Cross-entropy), 회기 문제에서는 평균 제곱 오차, 시퀀스 학습 문제에서는 CTC(Connection Temporal Classification) 등을 사용합니다. 완전히 새로운 연구를 할 때만 독자적인 목적 함수를 사용합니다.

## 3.2 케라스 소개

1. 입력 텐서와 타깃 텐서로 이루어진 훈련 데이터를 정의합니다.
2. 입력과 타깃을 매핑하는 층으로 이루어진 네트워크 (또는 모델)을 정의합니다.
3. 손실함수, 옵티마이저 , 모니터링하기 위한 측정 지표를 선택하여 학습과정을 설명합니다.
4. 훈련 데이터에 대해 모델의 `fit()`메서드를 반복적으로 호출합니다.

**모델을 정의하는 2가지 방법**
1. `Sequential`클래스를 이용한다. (가장 자주 사용하는 구조인 층을 순서대로 쌓아 올린 네트워크)
2. *함수형 API*를 사용한다. (완전히 임의의 구조를 만들 수 있는 비순환 유향 그래프)

In [2]:
### 1번째 방식
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape = (784, )))
model.add(layers.Dense(10, activation='softmax'))

In [3]:
### 2번째 방식
input_tensor = layers.Input(shape=(784, ))
x = layers.Dense(32, activation = 'relu')(input_tensor)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs = input_tensor, outputs = output_tensor)

In [4]:
### 모델 구현 이후에 어떤 방식을 사용했는지 상관없이 이후 단계는 동일하다.
from keras import optimizers

model.compile(optimizer = optimizers.RMSprop(lr=0.001),
             loss = 'mse',
             metrics = ['accuracy'])

In [5]:
model.fit(input_tensor, target_tensor, batch_size = 128, epochs = 10)

NameError: name 'target_tensor' is not defined

## 3.4 영화 리뷰 분류 : 이진 분류 예제


### 데이터 셋 준비

In [8]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

In [9]:
print(train_data[0], train_labels[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 1


In [10]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

1646592/1641221 [==============================] - 2s 1us/step


In [17]:
import numpy as np

def vectorize_sequences(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [18]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [19]:
print(x_train[0])

[0. 1. 1. ... 0. 0. 0.]


In [24]:
y_train = np.asarray(train_labels).astype('float32')
y_test  = np.asarray(test_labels).astype('float32')

### 신경망 모델 만들기

In [30]:
model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape=(10000, )))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

model.fit(x_train, y_train, epochs = 20, batch_size = 512)

Epoch 1/20
25000/25000 [==============================] - 3s 123us/step - loss: 0.4628 - acc: 0.8201
Epoch 2/20
25000/25000 [==============================] - 2s 86us/step - loss: 0.2653 - acc: 0.9083
Epoch 3/20
25000/25000 [==============================] - 2s 84us/step - loss: 0.2050 - acc: 0.9269
Epoch 4/20
25000/25000 [==============================] - 2s 87us/step - loss: 0.1702 - acc: 0.9404
Epoch 5/20
25000/25000 [==============================] - 2s 89us/step - loss: 0.1492 - acc: 0.9471
Epoch 6/20
25000/25000 [==============================] - 2s 81us/step - loss: 0.1288 - acc: 0.9568
Epoch 7/20
25000/25000 [==============================] - 2s 84us/step - loss: 0.1129 - acc: 0.9618
Epoch 8/20
25000/25000 [==============================] - 2s 90us/step - loss: 0.0980 - acc: 0.9684
Epoch 9/20
25000/25000 [==============================] - 2s 89us/step - loss: 0.0862 - acc: 0.9735
Epoch 10/20
25000/25000 [==============================] - 2s 82us/step - loss: 0.0755 - acc: 0.976